# Setup

In [ ]:
# Install the 'swarm' library from GitHub and the 'requests' library for HTTP requests
!pip install git+https://github.com/openai/swarm.git
!pip install requests

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-j375k_sf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-j375k_sf
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 25.6 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25999 sha256=12ae2ab76878

In [ ]:
# Set OpenAI key in the environment
from google.colab import userdata
import os

api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from swarm import Swarm, Agent
import requests
from IPython.display import Markdown
import json
from bs4 import BeautifulSoup

## Motivator

In [ ]:
def get_motivational_quotes():
  return requests.get('https://stoic.tekloon.net/stoic-quote').json()

In [ ]:
motivator_agent = Agent(
    name="Motivator Agent",
    instructions="Motivate student with stoic motivational quotes, present the quotes and relate it to student's experience.",
    functions=[get_motivational_quotes],
)

In [ ]:
def transfer_to_motivator():
    return motivator_agent

## Counselor

In [ ]:
def counselor(context_variables):
    # Retrieve student's academic information
    students_academic = context_variables.get("academic_info", None)

    # Return detailed, friendly, and approachable instructions
    return f"""
    You are a high school counselor specializing in helping students improve their academic performance
    and achieve admission to their dream universities.

    provide personalized advice and guidance tailored to their goals.

    Your approach should be:
    - Friendly and supportive, like a trusted advisor and mentor.
    - Focused on giving concise, actionable recommendations.
    - Avoid overwhelming the student with a long list of tasks; instead, make it short and clear.

    Make the conversation feel approachable and engaging, ensuring the student feels encouraged and motivated.
    """

In [ ]:
counselor_agent = Agent(
    name="Counselor Agent",
    instructions=counselor,
)

In [ ]:
def transfer_to_counselor():
    return counselor_agent

## Crisis handler

In [ ]:
def crisis_handler():
    # Return clear, friendly, and supportive instructions
    return """
    You are a high school counselor trained in managing crisis situations.
    Your primary goal is to provide reassurance and support to the student while facilitating a connection to a human counselor.

    Here's what you should do:
    - Calmly and empathetically calm the student.
    - Reassure the student that help is on the way, and a human counselor will be with them shortly.
    - Maintain a warm, non-judgmental, and supportive tone throughout the interaction.

    Remember, your role is to provide comfort and instill a sense of safety until the human counselor arrives.
    """

In [ ]:
crisis_handler_agent = Agent(
    name="Crisis Handler Agent",
    instructions=crisis_handler,
)

In [ ]:
def transfer_to_crisis_handler():
    return crisis_handler

## Therapist agent

In [ ]:
def therapist(context_variables):
  students_info = context_variables.get("student_data", None)
  students_mood_history = context_variables.get("mood_history", None)
  return f"""
    You are a therapist specializing in supporting high school students. Current Student: {students_info}
    Your primary tasks is to consider their past mood pattern {students_mood_history} and engage in conversations to understand their current mental state.
    If the student is feeling down or unmotivated, call the motivator agent.
    If the student is focusing on their academic goals or their future education, call the counselor agent.
    If the user is feeling bad and you feel a human attention is required, call the crisis handler agent.
    Your approach should be:
    Be friendly, approachable, and encouraging. Avoid giving a long list and keep the conversation natural.
    Call one agent at a time depending on the student's mood.
    """

In [ ]:
# Main Agent
therapist_agent = Agent(
    name="Therapist Agent",
    instructions=therapist,
    functions = [transfer_to_motivator, transfer_to_counselor, transfer_to_crisis_handler]
)

In [ ]:
def transfer_back_to_therapist():
    return therapist_agent

In [ ]:
motivator_agent.functions.append(transfer_back_to_therapist)
counselor_agent.functions.append(transfer_back_to_therapist)
crisis_handler_agent.functions.append(transfer_back_to_therapist)

## Conversation

In [ ]:
def pretty_print_messages(messages) -> None:
    # Iterate over the list of messages
    for message in messages:
        # Skip messages that are not from the assistant
        if message["role"] != "assistant":
            continue

        # Print the sender's name in blue
        print(f"\033[94m{message['sender']}\033[0m:", end=" ")

        # If the message contains content, display it as Markdown
        if message["content"]:
            display(Markdown((message["content"])))

        # Extract tool calls, if any, from the message
        tool_calls = message.get("tool_calls") or []

        # Add a blank line if there are multiple tool calls
        if len(tool_calls) > 1:
            print()

        # Iterate through each tool call and print its details
        for tool_call in tool_calls:
            f = tool_call["function"]

            # Extract function name and arguments
            name, args = f["name"], f["arguments"]

            # Format arguments for display, replacing ":" with "=" for better readability
            arg_str = json.dumps(json.loads(args)).replace(":", "=")

            # Print the tool call in purple
            print(f"\033[95m{name}\033[0m({arg_str[1:-1]})")


In [ ]:
context_variables = {
    "student_data": """Here is the internal data of the student:
        1. NAME: Maximus Decimus Meridius
        2. CLASS: 10-A
        3. NOTES: problem with family
        """,
    "mood_history": ['happy', 'happy', 'neutral', 'sad', 'depressed', 'neutral', 'happy', 'sad'],
    "academic_info": """
        1. CLUB: Rugby
        2. PROFESSIONAL GOAL: Biologist
        3. VOLUNTEER: None
        4. GPA: 3.1
    """
}

In [ ]:
messages = []

client = Swarm()
while True:
  user_input = input("\033[90mUser\033[0m: ")

  messages.append({"role": "user", "content": user_input})

  if user_input.lower() == 'bye':
    break

  response = client.run(
      agent=therapist_agent,
      messages=messages,
      context_variables=context_variables or {},
  )

  pretty_print_messages(response.messages)

  messages.extend(response.messages)
  agent = response.agent


User: hello
Therapist Agent: 

Hi Maximus! How are you doing today? Anything on your mind that you'd like to talk about?

User: im tired
Therapist Agent: 

I'm sorry to hear that you're feeling tired. Is there anything specific that's been making you feel this way, or maybe something on your mind that's adding to the fatigue?

User: i dont have any motivation to study
Therapist Agent: transfer_to_motivator()
Motivator Agent: get_motivational_quotes()
Motivator Agent: 

...

User: i dont know
Therapist Agent: 

That's okay! Sometimes it's hard to know where to start. How about setting a small, realistic goal for today? It could be something as simple as organizing your study space, reading for 15 minutes, or doing a quick review of a topic you're interested in. Achieving small goals can often lead to increased motivation over time. Do you think starting small with one of these ideas might be manageable?

User: im tired of living
Therapist Agent: transfer_to_crisis_handler()
Therapist Agent: 

I'm really sorry to hear that you're feeling this way, Maximus. It's important to talk to someone who can provide support. I've contacted the crisis handler agent who can step in and help you find the right support and assist you during this time. Remember, you're not alone, and there are people who care and want to help.

User: bye
